In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from pprint import pprint
import time
import re
from datetime import datetime
import os
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 리팩토링 코드

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from pprint import pprint
import time
import re
from datetime import datetime
import os
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# (이전 코드와 동일하므로 생략)
def process_game_price_data(data):
    processed_data = data.copy()

    # '원가'와 '할인가'를 정수형으로 변환
    for key in ['원가', '할인가']:
        if key in processed_data and isinstance(processed_data[key], str):
            # '₩'와 ',' 제거
            cleaned_value = re.sub(r'[₩,]', '', processed_data[key])
            try:
                processed_data[key] = int(cleaned_value)
            except ValueError:
                print(f"경고: '{key}' 값을 정수로 변환할 수 없습니다: {processed_data[key]}")
                processed_data[key] = None

    
    # '할인 시작일'를 'YYYY-MM-DD' 형식으로 변환
    if '할인 시작일' in processed_data and isinstance(processed_data['할인 시작일'], str):
        try:
            # 원본일 형식에 맞게 파싱
            date_obj = datetime.strptime(processed_data['할인 시작일'], '%d %b %Y, %H:%M')
            processed_data['할인 시작일'] = date_obj.strftime('%Y-%m-%d')
        except ValueError:
            print(f"경고: '할인 시작일' 형식을 변환할 수 없습니다: {processed_data['할인 시작일']}")
            processed_data['할인 시작일'] = None
    
    return processed_data

def history_log(game_name):
    # 1. game_name 유효성 검사
    if not game_name or not isinstance(game_name, str):
        print("경고: 유효하지 않은 game_name이 입력되었습니다. 'nodata'를 반환합니다.")
    
        return 'nodata'

    history_url = f"https://isthereanydeal.com/game/{game_name}/history/"
    # print(history_url)

    driver = None # 드라이버 초기화
    try:
        # Chrome 설정
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # 창 안띄우기
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(history_url)

        time.sleep(3) # 페이지 로드를 위한 대기

        # 2. 'nodata' 상황 판단: 특정 요소가 존재하지 않는지 확인
        # 'isthereanydeal.com'에서 게임이 없거나 페이지를 찾을 수 없을 때 나타나는 요소나 패턴을 확인합니다.
        # 예를 들어, "Page Not Found" 텍스트나, 데이터가 없는 경우에만 나타나는 특정 div를 찾습니다.
        # 여기서는 history-placeholder 클래스를 가진 div가 있는지 확인합니다.
        try:
            # 게임이 없을 경우 'There are currently no deals for this game.' 메시지를 포함하는
            # div.history-placeholder 요소가 나타납니다.
            msg_div = driver.find_element(By.CSS_SELECTOR, 'div.msg.svelte-jsvkvf')
            if "We didn't find the page you were looking for" in msg_div.text:
                print(f"'{game_name}'에 대한 URL이 존재하지 않습니다. 'nodata'를 반환합니다.")
                return 'nodata'
        except NoSuchElementException:
            # msg_div가 없으면, 게임 데이터가 있을 수 있다는 의미입니다. 계속 진행합니다.
            pass
        
        # 실제 게임 로그 데이터를 담고 있는 div 요소들을 가져옵니다.
        game_log = []
        div_tags = driver.find_elements(By.CSS_SELECTOR, 'div.entry.svelte-17mbxpf')

        if not div_tags: # div_tags가 비어있으면, 게임 데이터가 없는 것으로 판단합니다.
            print(f"'{game_name}'에 대한 거래 기록 div를 찾을 수 없습니다. 'nodata'를 반환합니다.")
        
            return 'nodata'

        for div_tag in div_tags:
            try:
                time_tag = div_tag.find_elements(By.CSS_SELECTOR, 'span.time.svelte-17mbxpf')
                discount_date = time_tag[0].text if time_tag else None

                store_tag = div_tag.find_elements(By.CSS_SELECTOR, 'span.svelte-zbwnbn')
                store = store_tag[0].text if store_tag else None

                cut_tag = div_tag.find_elements(By.CSS_SELECTOR, 'span.cut.svelte-17mbxpf')
                # cut_tag[0]은 할인의 종류 (e.g., "Discount"), cut_tag[1]이 할인율을 나타냅니다.
                cut = cut_tag[1].text if len(cut_tag) > 1 else None

                price_tag = div_tag.find_elements(By.CSS_SELECTOR, 'div.price.svelte-17mbxpf div')
                reg_price = price_tag[0].text if len(price_tag) > 0 else None
                discount_price = price_tag[2].text if len(price_tag) > 2 else None

                log = {
                    '게임 이름' : game_name,
                    '할인 시작일': discount_date,
                    '플랫폼 이름': store,
                    '할인율': cut,
                    '원가': reg_price,
                    '할인가': discount_price,

                }
                processed_log = process_game_price_data(log)
                game_log.append(processed_log)
            except Exception as e:
                print(f"개별 로그 처리 중 오류 발생: {e}")
            
                continue # 오류가 발생해도 다음 로그로 넘어갑니다.

        return game_log

    except WebDriverException as e:
        print(f"웹 드라이버 오류 발생: {e}. 'nodata'를 반환합니다.")
    
        return 'nodata'
    except Exception as e:
        print(f"예기치 않은 오류 발생: {e}. 'nodata'를 반환합니다.")
        
        return 'nodata'
    finally:
        if driver:
            time.sleep(2)
            driver.quit() # 오류 발생 여부와 상관없이 드라이버 종료

# data디렉토리를 생성하고 data_json 디렉토리에 파일을 넣어주는 함수
def save_json(data, game_name):
    """JSON 파일 저장 함수"""
    if data == 'nodata': # 'nodata'가 반환되면 파일 저장하지 않음
        print(f"'{game_name}'에 대한 데이터가 없어 JSON 파일을 저장하지 않습니다.")
        return

    os.makedirs('data/data_json', exist_ok=True)
    filepath = f'data/data_json/{game_name}.json'
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    # print(f"'{game_name}.json' 파일이 성공적으로 저장되었습니다.")

def jsonfile_to_csv(game_name):
    json_filepath = f'data/data_json/{game_name}.json'
    if not os.path.exists(json_filepath): # JSON 파일이 없으면 CSV 변환하지 않음
        # print(f"'{game_name}.json' 파일이 존재하지 않아 CSV로 변환할 수 없습니다.")
        return

    gamelog_df = pd.read_json(json_filepath)
    
    columns_to_convert = ['원가', '할인가', '이전과 비교한 할인가격', '할인 시작일', '할인율']
    for col in columns_to_convert:
        if col in gamelog_df.columns:
            # 먼저 숫자가 아닌 값(결측값 포함)을 'NoData'로 채우기 전에
            # 해당 열을 object 타입으로 변환합니다.
            # .astype(str) 대신 .astype(object)를 사용하는 것이 더 유연합니다.
            gamelog_df[col] = gamelog_df[col].astype(object)
    gamelog_df.fillna('NoData',inplace = True)
    os.makedirs('data/data_csv', exist_ok=True)
    gamelog_df.to_csv(f'data/data_csv/{game_name}.csv', index=False)
    # print(f"'{game_name}.csv' 파일이 성공적으로 생성되었습니다.")
    return

#csv파일에 있는 '게임 이름'을 사이트로 들어갈 수 있도록 처리
def clean_game_name_final(name):
    """
    게임 이름 문자열을 최종 클리닝하는 함수:
    - 모두 소문자로 변환
    - '&'를 'and'로 변환
    - 띄어쓰기를 하이픈으로 변환
    - 영어 소문자, 숫자, 하이픈 외 모든 문자 제거
    - 연속된 하이픈 합치기 및 불필요한 하이픈 제거
    """
    # 0. 입력값이 NaN일 경우 빈 문자열로 처리
    if pd.isna(name):
        return ""
        
    # 문자열로 변환하고 모두 소문자로 변경
    cleaned_name = str(name).lower()

    # 1. '&' 기호를 'and'로 변환 (먼저 처리)
    cleaned_name = cleaned_name.replace('&', 'and')

    # 2. 띄어쓰기(' ')를 하이픈('-')으로 변환
    # 이 단계를 모든 비-a-z0-9- 문자 제거 전에 수행하여 공백이 하이픈으로 변환되도록 합니다.
    cleaned_name = cleaned_name.replace(' ', '-')

    # 3. 영어 소문자, 숫자, 하이픈('-')을 제외한 모든 문자 제거
    # 이제 공백은 하이픈으로 변환되었으므로, 이 정규식은 하이픈을 제외한 나머지 불필요한 문자만 제거합니다.
    cleaned_name = re.sub(r'[^a-z0-9-]', '', cleaned_name)

    # 4. 연속으로 나타나는 하이픈을 하나로 줄이기 (예: 'metal--gear' -> 'metal-gear')
    # 이전 단계에서 공백이 하이픈으로 바뀌고, 다른 문자들이 제거되면서 연속 하이픈이 생길 수 있습니다.
    cleaned_name = re.sub(r'-+', '-', cleaned_name)
    
    # 5. 문장 시작/끝에 불필요하게 붙는 하이픈 제거 (예: '-metal-gear-' -> 'metal-gear')
    cleaned_name = cleaned_name.strip('-')
    
    # 6. 마지막으로, 혹시 남아있을 수 있는 공백 제거 (trim)
    cleaned_name = cleaned_name.strip()

    return cleaned_name


#각 게임의 로그 데이터에서 csv 파일 합치기
def merge_csv_files(folder_path, output_filename="merged_data.csv"):
    """
    지정된 폴더 내의 모든 CSV 파일을 하나로 합치고 새로운 CSV 파일로 저장합니다.

    Args:
        folder_path (str): CSV 파일들이 위치한 폴더의 경로.
        output_filename (str): 합쳐진 데이터를 저장할 CSV 파일의 이름.
    """
    all_files = []
    # 폴더 내의 모든 파일 목록을 가져옵니다.
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                all_files.append(os.path.join(root, file))

    if not all_files:
        print(f"경로 '{folder_path}'에 CSV 파일이 없습니다.")
        return

    # 첫 번째 파일을 읽어서 컬럼 구조를 확인하거나,
    # 단순히 모든 파일을 읽고 concat 합니다.
    # 여기서는 모든 파일을 읽어서 합치는 일반적인 방법을 사용합니다.
    df_list = []
    for f in all_files:
        try:
            df = pd.read_csv(f)
            df_list.append(df)
            # print(f"파일 로드 성공: {f}")
        except Exception as e:
            print(f"파일 로드 실패: {f} - {e}")
            continue

    if not df_list:
        print("로드에 성공한 CSV 파일이 없습니다. 병합 작업을 수행할 수 없습니다.")
        return

    # 모든 DataFrame을 하나로 합칩니다.
    # ignore_index=True를 설정하여 합칠 때 인덱스가 겹치지 않도록 새로 생성합니다.
    merged_df = pd.concat(df_list, ignore_index=True)

    # 합쳐진 DataFrame을 새로운 CSV 파일로 저장합니다.
    output_path = os.path.join(folder_path, output_filename)
    merged_df.to_csv(output_path, index=False) # index=False는 DataFrame의 인덱스를 CSV에 저장하지 않도록 합니다.

    print(f"\n총 {len(all_files)}개의 CSV 파일이 성공적으로 병합되었습니다.")
    print(f"병합된 파일은 '{output_path}'에 저장되었습니다.")

if __name__ == "__main__":
    # 게임 csv파일을 받아서 '게임 이름'을 list로 받기
    csvfilename = 'cleaned_merged_games_data.csv' #합쳐진 데이터로 바꾸기(경로 변경)
    game_data = pd.read_csv(csvfilename)

    game_data['게임 이름'] = game_data['게임 이름'].apply(clean_game_name_final)
    games = list(set(game_data['게임 이름'].tolist()))
    nogamelog=list()
    trylog1 = list()
    trylog2 = list()

    #gamelog를 json 및 csv 파일로 저장
    for game_name in games[:3]:
        gamelog = history_log(game_name)
        if gamelog != 'nodata':
            save_json(gamelog, game_name)
            jsonfile_to_csv(game_name)
            json_file_name = f'{game_name}.json'
            os.remove(os.path.join('data/data_json', json_file_name))
        else:
            print(f"'{game_name}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")
            nogamelog.append(game_name)
        
    print('첫번째 시도 : data를 찾을 수 없었던 파일에 한하여 다시 시도합니다')
    for game_name in nogamelog:
        gamelog = history_log(game_name)
        if gamelog != 'nodata':
            save_json(gamelog, game_name)
            jsonfile_to_csv(game_name)
            os.remove(os.path.join('data/data_json', json_file_name))
        else:
            print(f"'{game_name}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")
            trylog1.append(game_name)

    print('두번째 시도 : data를 찾을 수 없었던 파일에 한하여 다시 시도합니다')
    for game_name in trylog1:
        gamelog = history_log(game_name)
        if gamelog != 'nodata':
            save_json(gamelog, game_name)
            jsonfile_to_csv(game_name)
            os.remove(os.path.join('data/data_json', json_file_name))
        else:
            print(f"'{game_name}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")
            trylog2.append(game_name)

    with open('data/game_name.txt','w') as f:
        for game_name in trylog2:
            f.write(game_name+'\n')
        print('로그 데이터를 찾을 수 없는 게임을 gamename.txt에 저장하였습니다')
   
    #각 게임의 csv파일 합치기
    my_folder = "data/data_csv"
    csv_folder_path = my_folder
    merge_csv_files(csv_folder_path, "../combined_sales_data.csv")

### 게임 기본 정보에 저장된 게임이름을 뽑아낸 후 각 게임의 log데이터 만들기

In [ ]:
# 게임 csv파일을 받아서 '게임 이름'을 list로 받기
csvfilename = 'merged_games_data.csv' #합쳐진 데이터로 바꾸기(경로 변경)
game_data = pd.read_csv(csvfilename)

game_data['게임 이름'] = game_data['게임 이름'].apply(clean_game_name_final)
games = list(set(game_data['게임 이름'].tolist()))
nogamelog=list()

#gamelog를 json 및 csv 파일로 저장
for game_name in games :
    gamelog = history_log(game_name)
    if gamelog != 'nodata':
        nogamelog.append(game_name)
        save_json(gamelog, game_name)
        jsonfile_to_csv(game_name)
    else:
        print(f"'{game_name}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")



In [3]:

#각 게임의 csv파일 합치기
my_folder = "data/data_csv"
csv_folder_path = my_folder
merge_csv_files(csv_folder_path, "../combined_sales_data.csv")


총 276개의 CSV 파일이 성공적으로 병합되었습니다.
병합된 파일은 'data/data_csv\../combined_sales_data.csv'에 저장되었습니다.


In [9]:
data = pd.read_csv('data/data_csv/ostranauts.csv')

# game_name = data[data['게임 이름']=='dave-the-diver'].copy()
data
# visualize(game_name)


,게임 이름,할인 시작일,플랫폼 이름,할인율,원가,할인가
0,ostranauts,2025-07-11,Steam,0%,21500,21500
1,ostranauts,2025-06-27,Steam,20%,21500,17200
2,ostranauts,2025-05-24,Steam,0%,21500,21500
3,ostranauts,2025-05-17,Steam,20%,21500,17200
4,ostranauts,2025-04-02,Steam,0%,21500,21500
5,ostranauts,2025-03-29,Steam,20%,21500,17200
6,ostranauts,2025-03-21,Steam,0%,21500,21500
7,ostranauts,2025-03-14,Steam,20%,21500,17200
8,ostranauts,2025-01-31,Steam,0%,21500,21500
9,ostranauts,2025-01-24,Steam,20%,21500,17200


### csv 파일 합치기

In [ ]:
# import pandas as pd
# import os

# def merge_csv_files(folder_path, output_filename="merged_data.csv"):
#     """
#     지정된 폴더 내의 모든 CSV 파일을 하나로 합치고 새로운 CSV 파일로 저장합니다.

#     Args:
#         folder_path (str): CSV 파일들이 위치한 폴더의 경로.
#         # output_filename (str): 합쳐진 데이터를 저장할 CSV 파일의 이름.
#     """
#     all_files = []
#     # 폴더 내의 모든 파일 목록을 가져옵니다.
#     for root, dirs, files in os.walk(folder_path):
#         for file in files:
#             if file.endswith('.csv'):
#                 all_files.append(os.path.join(root, file))

#     if not all_files:
#         print(f"경로 '{folder_path}'에 CSV 파일이 없습니다.")
#         return

#     # 첫 번째 파일을 읽어서 컬럼 구조를 확인하거나,
#     # 단순히 모든 파일을 읽고 concat 합니다.
#     # 여기서는 모든 파일을 읽어서 합치는 일반적인 방법을 사용합니다.
#     df_list = []
#     for f in all_files:
#         try:
#             df = pd.read_csv(f)
#             df_list.append(df)
#             print(f"파일 로드 성공: {f}")
#         except Exception as e:
#             print(f"파일 로드 실패: {f} - {e}")
#             continue

#     if not df_list:
#         print("로드에 성공한 CSV 파일이 없습니다. 병합 작업을 수행할 수 없습니다.")
#         return

#     # 모든 DataFrame을 하나로 합칩니다.
#     # ignore_index=True를 설정하여 합칠 때 인덱스가 겹치지 않도록 새로 생성합니다.
#     merged_df = pd.concat(df_list, ignore_index=True)

#     # 합쳐진 DataFrame을 새로운 CSV 파일로 저장합니다.
#     output_path = os.path.join(folder_path, output_filename)
#     merged_df.to_csv(output_path, index=False) # index=False는 DataFrame의 인덱스를 CSV에 저장하지 않도록 합니다.

#     print(f"\n총 {len(all_files)}개의 CSV 파일이 성공적으로 병합되었습니다.")
#     print(f"병합된 파일은 '{output_path}'에 저장되었습니다.")

# # --- 사용 예시 ---
# if __name__ == "__main__":
#     my_folder = "data/data_csv"
#     csv_folder_path = my_folder 
#     merge_csv_files(csv_folder_path, "../combined_sales_data.csv")

### 시각화

In [ ]:
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go


# # '할인 시작 날짜'를 datetime 형식으로 변환
# game_data['할인 시작 날짜'] = pd.to_datetime(game_data['할인 시작 날짜'])

# # Plotly를 사용하여 산점도 생성
# fig = px.scatter(game_data,
#                  x='할인 시작 날짜',
#                  y='할인가',
#                  hover_name='플랫폼 이름',
#                  hover_data={'할인가': ':,'},  # 할인가를 쉼표와 함께 전체 숫자로 표시
#                  title='날짜별 할인가 및 최저가 추이',
#                  labels={'할인 시작 날짜': '할인 시작 날짜', '할인가': '할인가 (원)'})

# # 각 날짜별 최저가 계산
# game_data_min_price = game_data.groupby('할인 시작 날짜')['할인가'].min().reset_index()
# game_data_min_price.columns = ['할인 시작 날짜', '최저 할인가']

# # 최저가를 나타내는 라인 트레이스 추가
# fig.add_trace(
#     go.Scatter(
#         x=game_data_min_price['할인 시작 날짜'],
#         y=game_data_min_price['최저 할인가'],
#         mode='lines+markers',
#         name='일별 최저 할인가',
#         line=dict(color='red', width=2),
#         marker=dict(size=8, symbol='circle-open', color='red'),
#         hovertemplate='<b>날짜:</b> %{x|%Y-%m-%d}<br><b>최저 할인가:</b> %{y:,}원<extra></extra>' # 쉼표와 함께 전체 숫자로 표시
#     )
# )

# # 그래프 레이아웃 업데이트
# fig.update_traces(marker=dict(size=10, opacity=0.8),
#                   selector=dict(mode='markers'))

# fig.update_layout(xaxis_title="할인 날짜",
#                   yaxis_title="할인가",
#                   xaxis_tickformat='%Y-%m-%d',
#                   yaxis_tickformat=',', # y축 레이블을 쉼표를 포함한 전체 숫자로 표시
#                   legend_title_text='범례')

# fig.show()

In [ ]:
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def visualize(game_data):
#     # '할인 시작 날짜'를 datetime 형식으로 변환
#     game_data['할인 시작 날짜'] = pd.to_datetime(game_data['할인 시작 날짜'])

#     # Plotly를 사용하여 산점도 생성
#     fig = px.scatter(game_data,
#                     x='할인 시작 날짜',
#                     y='할인가',
#                     hover_name='플랫폼 이름',
#                     hover_data={'할인가': ':,'},  # 할인가를 쉼표와 함께 전체 숫자로 표시
#                     title='날짜별 할인가 및 최저가 추이',
#                     labels={'할인 시작 날짜': '할인 시작 날짜', '할인가': '할인가 (원)'})

#     # 각 날짜별 최저가 계산
#     game_data_min_price = game_data.groupby('할인 시작 날짜')['할인가'].min().reset_index()
#     game_data_min_price.columns = ['할인 시작 날짜', '최저 할인가']

#     # 최저가를 나타내는 라인 트레이스 추가
#     fig.add_trace(
#         go.Scatter(
#             x=game_data_min_price['할인 시작 날짜'],
#             y=game_data_min_price['최저 할인가'],
#             mode='lines+markers',
#             name='일별 최저 할인가',
#             line=dict(color='red', width=2),
#             marker=dict(size=8, symbol='circle-open', color='red'),
#             hovertemplate='<b>날짜:</b> %{x|%Y-%m-%d}<br><b>최저 할인가:</b> %{y:,}원<extra></extra>' # 쉼표와 함께 전체 숫자로 표시
#         )
#     )

#     # 그래프 레이아웃 업데이트
#     fig.update_traces(marker=dict(size=10, opacity=0.8),
#                     selector=dict(mode='markers'))

#     fig.update_layout(xaxis_title="할인 날짜",
#                     yaxis_title="할인가",
#                     xaxis_tickformat='%Y-%m-%d',
#                     yaxis_tickformat=',', # y축 레이블을 쉼표를 포함한 전체 숫자로 표시
#                     legend_title_text='범례')

#     fig.show()
#     return

### 안 될 경우 

In [ ]:

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #Chrome을 위한 webdriver install
# driver.get("https://isthereanydeal.com/game/call-of-duty-modern-warfare-ii/history/")

# time.sleep(5)

# #div 엘리먼트 선택해서 가져오기
# div_tags = driver.find_elements(By.CSS_SELECTOR,'div.entry.svelte-17mbxpf')
# print(len(div_tags))

# for div_tag in div_tags :
#     time_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.time.svelte-17mbxpf')
#     discount_date = time_tag[0].text
#     print(discount_date)

#     store_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.svelte-zbwnbn')
#     store = store_tag[0].text

#     duration_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.duration.svelte-17mbxpf')
#     duration = duration_tag[0].text

#     cut_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.cut.svelte-17mbxpf')
#     cut = cut_tag[1].text

#     price_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.price.svelte-17mbxpf div')
#     reg_price = price_tag[0].text
#     discount_price = price_tag[2].text
#     price_comp = price_tag[3].text

#     log = {'할인 시작 날짜':discount_date, '플랫폼 이름': store,'할인 지속시간':duration,'할인율':cut,'원가':reg_price,'할인가':discount_price,'이전과 비교한 할인가격':price_comp}
#     pprint(log)
#     # processed_log=process_price_data_with_sign(log)

# #5초로 waiting time 설정
# time.sleep(5)
# #driver 종료
# driver.quit()



In [ ]:
# import pandas as pd
# import re

# def clean_game_name_final(name):
#     """
#     게임 이름 문자열을 최종 클리닝하는 함수:
#     - 모두 소문자로 변환
#     - '&'를 'and'로 변환
#     - 띄어쓰기를 하이픈으로 변환
#     - 영어 소문자, 숫자, 하이픈 외 모든 문자 제거
#     - 연속된 하이픈 합치기 및 불필요한 하이픈 제거
#     """
#     # 0. 입력값이 NaN일 경우 빈 문자열로 처리
#     if pd.isna(name):
#         return ""
        
#     # 문자열로 변환하고 모두 소문자로 변경
#     cleaned_name = str(name).lower()

#     # 1. '&' 기호를 'and'로 변환 (먼저 처리)
#     cleaned_name = cleaned_name.replace('&', 'and')

#     # 2. 띄어쓰기(' ')를 하이픈('-')으로 변환
#     # 이 단계를 모든 비-a-z0-9- 문자 제거 전에 수행하여 공백이 하이픈으로 변환되도록 합니다.
#     cleaned_name = cleaned_name.replace(' ', '-')

#     # 3. 영어 소문자, 숫자, 하이픈('-')을 제외한 모든 문자 제거
#     # 이제 공백은 하이픈으로 변환되었으므로, 이 정규식은 하이픈을 제외한 나머지 불필요한 문자만 제거합니다.
#     cleaned_name = re.sub(r'[^a-z0-9-]', '', cleaned_name)

#     # 4. 연속으로 나타나는 하이픈을 하나로 줄이기 (예: 'metal--gear' -> 'metal-gear')
#     # 이전 단계에서 공백이 하이픈으로 바뀌고, 다른 문자들이 제거되면서 연속 하이픈이 생길 수 있습니다.
#     cleaned_name = re.sub(r'-+', '-', cleaned_name)
    
#     # 5. 문장 시작/끝에 불필요하게 붙는 하이픈 제거 (예: '-metal-gear-' -> 'metal-gear')
#     cleaned_name = cleaned_name.strip('-')
    
#     # 6. 마지막으로, 혹시 남아있을 수 있는 공백 제거 (trim)
#     cleaned_name = cleaned_name.strip()

#     return cleaned_name
